In [1]:
%pip install scikit-learn
%pip install scipy
%pip install pandas
%pip install transformers
%pip install tensorflow
%pip install torch
%pip install tqdm
%pip install tensorflowjs[wizard]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached tensorflowjs-4.12.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached flax-0.7.4-py3-none-any.whl.metadata (10 kB)
  Using cached importlib_resources-6.1.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached jax-0.4.19-py3-none-any.whl.metadata (23 kB)
  Using cached jaxlib-0.4.19-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached tensorflow_decision_forests-1.6.0-cp39-cp39-macosx_12_0_arm64.whl.metadata (6.0 kB)
  Using cached tensorflow_hub-0.15.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached PyI

In [2]:
import tensorflow as tf
from transformers import TFAutoModel

MODEL_NAME = './MultitaskModel'
model = TFAutoModel.from_pretrained(MODEL_NAME, from_pt = True)

callable = tf.function(model.call)
concrete_function = callable.get_concrete_function([tf.TensorSpec([None, 384], tf.int32, name="input_ids"),tf.TensorSpec([None, 384], tf.int32, name="attention_mask")])

tf.saved_model.save(model, 'multitaskModelForJS_graphdef', signatures=concrete_function)
!saved_model_cli show --dir multitaskModelForJS_graphdef --tag_set serve --signature_def serving_default

! tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_format=tfjs_graph_model \
    --signature_name=serving_default \
    --saved_model_tags=serve \
    ./multitaskModelForJS_graphdef/ \
    ./multitaskModelForJSWeb_graphdef/

/Users/danielgoldelman/Desktop/tfjs_conv_issue/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/danielgoldelman/Desktop/tfjs_conv_issue/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'classifier.bias', 'classifier.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if

INFO:tensorflow:Assets written to: multitaskModelForJS_graphdef/assets


INFO:tensorflow:Assets written to: multitaskModelForJS_graphdef/assets


The given SavedModel SignatureDef contains the following input(s):
  inputs['attention_mask'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 384)
      name: serving_default_attention_mask:0
  inputs['input_ids'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 384)
      name: serving_default_input_ids:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['last_hidden_state'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 384, 312)
      name: StatefulPartitionedCall:0
  outputs['pooler_output'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 312)
      name: StatefulPartitionedCall:1
Method name is: tensorflow/serving/predict
/Users/danielgoldelman/Desktop/tfjs_conv_issue/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
import json
import sklearn
from sklearn import metrics
import scipy as sp
import pandas as pd

In [9]:
ppAllTestSet = pd.read_json(path_or_buf='ppAllTestSet.jsonl', lines=True)
ppAllTestSet = ppAllTestSet.set_index('id')
modelTestResults_B = pd.read_json(path_or_buf='modelTestResults_B.json')
modelTestResults_B = modelTestResults_B.set_index('id')
modelTestResults_B['result'] = modelTestResults_B['result'].apply(lambda x: 1 if x else 0)

In [11]:
print(metrics.classification_report(y_true=ppAllTestSet['label'],y_pred=modelTestResults_B['result'],target_names=['False','True']))

              precision    recall  f1-score   support

       False       0.88      0.94      0.91       290
        True       0.93      0.85      0.89       243

    accuracy                           0.90       533
   macro avg       0.91      0.90      0.90       533
weighted avg       0.90      0.90      0.90       533



In [12]:
for datatype in ['city', 'lat', 'lng', 'region', 'zip']:
  indexes = ppAllTestSet['data'].apply(lambda x: x.startswith(datatype))
  indexes = indexes[indexes].index
  pred = modelTestResults_B.filter(indexes, axis=0)['result']
  exp = ppAllTestSet.filter(indexes, axis=0)['label']
  print(f"---Classification Report for {datatype}---")
  print(metrics.classification_report(y_true=list(exp),y_pred=list(pred),target_names=['False','True']))

---Classification Report for city---
              precision    recall  f1-score   support

       False       0.85      0.95      0.90        63
        True       0.91      0.72      0.81        40

    accuracy                           0.86       103
   macro avg       0.88      0.84      0.85       103
weighted avg       0.87      0.86      0.86       103

---Classification Report for lat---
              precision    recall  f1-score   support

       False       0.76      0.93      0.83        40
        True       0.95      0.82      0.88        67

    accuracy                           0.86       107
   macro avg       0.85      0.87      0.86       107
weighted avg       0.88      0.86      0.86       107

---Classification Report for lng---
              precision    recall  f1-score   support

       False       0.80      0.93      0.86        44
        True       0.94      0.84      0.89        61

    accuracy                           0.88       105
   macro avg       

In [44]:
from transformers import pipeline, TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow
from tqdm import tqdm

In [37]:
location = "./MultitaskModel"
model = TFAutoModelForSequenceClassification.from_pretrained(location, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(location)
multitaskModel = pipeline('text-classification',model=model, tokenizer=tokenizer)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [38]:
multitaskModel("a")

[{'label': 'LABEL_0', 'score': 0.9998854398727417}]

In [39]:
print(multitaskModel(ppAllTestSet.iloc[0,:]["data"]))

[{'label': 'LABEL_0', 'score': 0.9999079704284668}]


In [45]:
t_All = []
multi_All = []
for i in tqdm(range(533)):
  t_All.append(ppAllTestSet.iloc[i,:]['label'])
  res = multitaskModel(ppAllTestSet.iloc[i,:]["data"])
  label = res[0]['label']
  if label == 'LABEL_0':
    multi_All.append(0)
  else:
    multi_All.append(1)
  
print(f"---Classification Report for All---")
print(metrics.classification_report(y_true=t_All,y_pred=multi_All,target_names=['False','True']))

100%|██████████| 533/533 [00:14<00:00, 36.28it/s]

---Classification Report for All---
              precision    recall  f1-score   support

       False       0.97      0.92      0.94       290
        True       0.91      0.96      0.94       243

    accuracy                           0.94       533
   macro avg       0.94      0.94      0.94       533
weighted avg       0.94      0.94      0.94       533



In [50]:
for datatype in ['city', 'lat', 'lng', 'region', 'zip']:
  t = []
  multi = []
  for i in range(533):
    if ppAllTestSet.iloc[i,:]["data"].startswith(datatype):
      t.append(ppAllTestSet.iloc[i,:]['label'])
      res = multitaskModel(ppAllTestSet.iloc[i,:]["data"])
      label = res[0]['label']
      if label == 'LABEL_0':
        multi.append(0)
      else:
        multi.append(1)
    
  print(f"---Classification Report for {datatype}---")
  print(metrics.classification_report(y_true=t,y_pred=multi,target_names=['False','True']))

---Classification Report for city---
              precision    recall  f1-score   support

       False       0.92      0.87      0.89        63
        True       0.81      0.88      0.84        40

    accuracy                           0.87       103
   macro avg       0.87      0.87      0.87       103
weighted avg       0.88      0.87      0.87       103

---Classification Report for lat---
              precision    recall  f1-score   support

       False       0.94      0.85      0.89        40
        True       0.92      0.97      0.94        67

    accuracy                           0.93       107
   macro avg       0.93      0.91      0.92       107
weighted avg       0.93      0.93      0.92       107

---Classification Report for lng---
              precision    recall  f1-score   support

       False       0.95      0.80      0.86        44
        True       0.87      0.97      0.91        61

    accuracy                           0.90       105
   macro avg       